In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

In [2]:
train = pd.read_csv('./train.csv.zip',compression='zip', 
                   dtype={'date': str, 'fullVisitorId': str, 'sessionId':str}, nrows=None)

test = pd.read_csv('./test.csv.zip',compression='zip', 
                   dtype={'date': str, 'fullVisitorId': str, 'sessionId':str}, nrows=None)


In [3]:
print(train.info())
train.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 12 columns):
channelGrouping         903653 non-null object
date                    903653 non-null object
device                  903653 non-null object
fullVisitorId           903653 non-null object
geoNetwork              903653 non-null object
sessionId               903653 non-null object
socialEngagementType    903653 non-null object
totals                  903653 non-null object
trafficSource           903653 non-null object
visitId                 903653 non-null int64
visitNumber             903653 non-null int64
visitStartTime          903653 non-null int64
dtypes: int64(3), object(9)
memory usage: 82.7+ MB
None


,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147


In [4]:
print(test.info())
test.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804684 entries, 0 to 804683
Data columns (total 12 columns):
channelGrouping         804684 non-null object
date                    804684 non-null object
device                  804684 non-null object
fullVisitorId           804684 non-null object
geoNetwork              804684 non-null object
sessionId               804684 non-null object
socialEngagementType    804684 non-null object
totals                  804684 non-null object
trafficSource           804684 non-null object
visitId                 804684 non-null int64
visitNumber             804684 non-null int64
visitStartTime          804684 non-null int64
dtypes: int64(3), object(9)
memory usage: 73.7+ MB
None


,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6167871330617112363,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",6167871330617112363_1508151024,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""4""}","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508151024,2,1508151024
1,Organic Search,20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",0643697640977915618,"{""continent"": ""Europe"", ""subContinent"": ""South...",0643697640977915618_1508175522,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508175522,1,1508175522


-----------
## Totals

In [5]:
totals_train = train.totals.copy()
totals_test  = test.totals.copy()

cols_train = ['bounces', 'hits', 'newVisits', 'pageviews','transactionRevenue']
cols_test = cols = ['bounces', 'hits', 'newVisits', 'pageviews']

totals_train_df = pd.DataFrame(columns=cols_train, data=np.zeros((totals_train.shape[0],len(cols_train))))
totals_test_df  = pd.DataFrame(columns=cols_test, data=np.zeros((totals_test.shape[0],len(cols_test))))


for t,c in zip(totals_train,range(totals_train.shape[0])):
    for i in t[1:-1].split(', ') :
        typ,val = [ j[1:-1] for j in i.split(': ')]
        totals_train_df.at[c,typ] = val

for t,c in zip(totals_test,range(totals_test.shape[0])):
    for i in t[1:-1].split(', ') :
        typ,val = [ j[1:-1] for j in i.split(': ')]
        totals_test_df.at[c,typ] = val
        
                
new_train = pd.concat((train,totals_train_df),sort=False,axis=1)
new_train.drop(columns=['totals'], inplace=True)

new_test = pd.concat((test,totals_test_df),sort=False,axis=1)
new_test.drop(columns=['totals'], inplace=True)

----------------

## TrafficSource

In [6]:
trafficTrain = train.trafficSource.copy()
trafficTest = test.trafficSource.copy()


In [7]:
cols = ['campaign', 'medium', 'adNetworkType', 'gclId', 'adwordsClickInfo', 'targetingCriteria',
        'isTrueDirect', 'adContent', 'isVideoAd', 'slot', 'campaignCode', 'criteriaParameters',
        'referralPath', 'source', 'keyword']

trafficTrain_df = pd.DataFrame(columns=cols, data=np.zeros((trafficTrain.shape[0],len(cols))),dtype=str)
trafficTest_df = pd.DataFrame(columns=cols, data=np.zeros((trafficTest.shape[0],len(cols))),dtype=str)


for t,c in zip(trafficTrain,range(trafficTrain.shape[0])):
    for i in t[1:-1].split(', '):
        try :
            typ,val = [ j[1:-1] for j in i.split(': ',1)]
            trafficTrain_df.at[c,typ] = val
        except:
            print(i,end=', ')
print('\n\n')

for t,c in zip(trafficTest,range(trafficTest.shape[0])):
    for i in t[1:-1].split(', '):
        try :
            typ,val = [ j[1:-1] for j in i.split(': ',1)]
            trafficTest_df.at[c,typ] = val
        except:
            print(i,end=', ')


CA", belfry t shirts", shirts, business card, cap,etx", CA", CA", CA", CA", 


Com", CA", hoodies caps", CA", boys", google", 

In [8]:
trafficTrain_df = trafficTrain_df[['medium','isTrueDirect','isVideoAd','slot','source']].copy()
trafficTest_df = trafficTest_df[['medium','isTrueDirect','isVideoAd','slot','source']].copy()

Lencoder = LabelEncoder()

_ = Lencoder.fit(pd.concat((trafficTrain_df.medium, trafficTest_df.medium)))
trafficTrain_df.medium = Lencoder.transform(trafficTrain_df.medium)
trafficTest_df.medium = Lencoder.transform(trafficTest_df.medium)

_ = Lencoder.fit(pd.concat((trafficTrain_df.isTrueDirect, trafficTest_df.isTrueDirect)))
trafficTrain_df.isTrueDirect = Lencoder.transform(trafficTrain_df.isTrueDirect)
trafficTest_df.isTrueDirect = Lencoder.transform(trafficTest_df.isTrueDirect)

_ = Lencoder.fit(pd.concat((trafficTrain_df.isVideoAd, trafficTest_df.isVideoAd)))
trafficTrain_df.isVideoAd = Lencoder.transform(trafficTrain_df.isVideoAd)
trafficTest_df.isVideoAd = Lencoder.transform(trafficTest_df.isVideoAd)

_ = Lencoder.fit(pd.concat((trafficTrain_df.slot, trafficTest_df.slot)))
trafficTrain_df.slot = Lencoder.transform(trafficTrain_df.slot)
trafficTest_df.slot = Lencoder.transform(trafficTest_df.slot)

# later I'll work on this
trafficTrain_df.drop(columns=['source'],inplace=True)
trafficTest_df.drop(columns=['source'],inplace=True)

trafficTest_df.head(3)

,medium,isTrueDirect,isVideoAd,slot
0,5,1,0,0
1,5,0,0,0
2,5,0,0,0


In [9]:
new_train = pd.concat((new_train,trafficTrain_df),sort=False,axis=1)
new_train.drop(columns=['trafficSource'], inplace=True)

new_test = pd.concat((new_test,trafficTest_df),sort=False,axis=1)
new_test.drop(columns=['trafficSource'], inplace=True)

In [28]:
train_bu = new_train.copy()
test_bu = new_test.copy()

-------

## ChannelGrouping, SocialEngagementType, Date

In [10]:
# ChannelGrouping
Lencoder = LabelEncoder()
_ = Lencoder.fit(pd.concat( (new_train.channelGrouping,new_test.channelGrouping)))
new_train.channelGrouping = Lencoder.transform(new_train.channelGrouping)
new_test.channelGrouping = Lencoder.transform(new_test.channelGrouping)

In [11]:
# SocialEngagementType
new_train.drop(columns=['socialEngagementType'], inplace=True)
new_test.drop(columns=['socialEngagementType'], inplace=True)

In [13]:
# Date
dateTrain = new_train.date.copy()
dateTrain_df = pd.DataFrame(columns=['year', 'month', 'dayOfYear'], data=np.zeros((dateTrain.shape[0],3)))

dateTest = new_test.date.copy()
dateTest_df = pd.DataFrame(columns=['year', 'month', 'dayOfYear'], data=np.zeros((dateTest.shape[0],3)))


for d,ind in zip(dateTrain,range(len(dateTrain))) :
    dateTrain_df.at[ind,'year'] = np.int(d[:4])
    dateTrain_df.at[ind,'month'] = np.int(d[4:6])
    adate = datetime.strptime(d, "%Y%m%d")
    dateTrain_df.at[ind,'dayOfYear'] = adate.timetuple().tm_yday
    
for d,ind in zip(dateTest,range(len(dateTest))) :
    dateTest_df.at[ind,'year'] = np.int(d[:4])
    dateTest_df.at[ind,'month'] = np.int(d[4:6])
    adate = datetime.strptime(d, "%Y%m%d")
    dateTest_df.at[ind,'dayOfYear'] = adate.timetuple().tm_yday

In [14]:
new_train = pd.concat((new_train,dateTrain_df),sort=False,axis=1)
new_train.drop(columns=['date'], inplace=True)

new_test = pd.concat((new_test,dateTest_df),sort=False,axis=1)
new_test.drop(columns=['date'], inplace=True)



--------------------------------------

## geoNetwork

In [20]:
geoNetworkTrain = new_train.geoNetwork.copy()
geoNetworkTest = new_test.geoNetwork.copy()

cols = ['continent', 'subContinent', 'country']
geoNetworkTrain_df = pd.DataFrame(data=np.zeros([len(geoNetworkTrain),len(cols)]),columns=cols, dtype=np.str)
geoNetworkTest_df = pd.DataFrame(data=np.zeros([len(geoNetworkTest),len(cols)]),columns=cols, dtype=np.str)

for t,c in zip(geoNetworkTrain,range(geoNetworkTrain.shape[0])):
    for i in t[1:-1].split(', ') :
        typ,val = [ j[1:-1] for j in i.split(': ')]
        if typ in cols :
            geoNetworkTrain_df.at[c,typ] = val
            
for t,c in zip(geoNetworkTest,range(geoNetworkTest.shape[0])):
    for i in t[1:-1].split(', ') :
        typ,val = [ j[1:-1] for j in i.split(': ')]
        if typ in cols :
            geoNetworkTest_df.at[c,typ] = val

In [21]:
lencoder = LabelEncoder()
_ = lencoder.fit(pd.concat((geoNetworkTrain_df.continent, geoNetworkTest_df.continent)))
geoNetworkTrain_df.continent = lencoder.transform(geoNetworkTrain_df.continent)
geoNetworkTest_df.continent = lencoder.transform(geoNetworkTest_df.continent)

_ = lencoder.fit(pd.concat((geoNetworkTrain_df.subContinent, geoNetworkTest_df.subContinent)))
geoNetworkTrain_df.subContinent = lencoder.transform(geoNetworkTrain_df.subContinent)
geoNetworkTest_df.subContinent = lencoder.transform(geoNetworkTest_df.subContinent)

_ = lencoder.fit(pd.concat((geoNetworkTrain_df.country, geoNetworkTest_df.country)))
geoNetworkTrain_df.country = lencoder.fit_transform(geoNetworkTrain_df.country)
geoNetworkTest_df.country = lencoder.fit_transform(geoNetworkTest_df.country)



In [22]:
new_train = pd.concat((new_train,geoNetworkTrain_df),sort=False,axis=1)
new_train.drop(columns=['geoNetwork'], inplace=True)

new_test = pd.concat((new_test,geoNetworkTest_df),sort=False,axis=1)
new_test.drop(columns=['geoNetwork'], inplace=True)


------------------------

## Device



In [24]:
deviceTrain = new_train.device.copy()
deviceTest = new_test.device.copy()

In [25]:

cols = ['browser', 'operatingSystem', 'deviceCategory']
deviceTrain_df = pd.DataFrame(data=np.zeros([len(deviceTrain),len(cols)]),columns=cols, dtype=np.str)
deviceTest_df = pd.DataFrame(data=np.zeros([len(deviceTest),len(cols)]),columns=cols, dtype=np.str)


for t,ind in zip(deviceTrain,range(deviceTrain.shape[0])):
    for i in t[1:-1].split(', ') :
        try:
            typ,val = [ j[1:-1] for j in i.split(': ')]
            if typ in cols :
                deviceTrain_df.at[ind,typ] = val
        except:
            print(i)
            
print('\n\n')
for t,ind in zip(deviceTest,range(deviceTest.shape[0])):
    for i in t[1:-1].split(', ') :
        try:
            typ,val = [ j[1:-1] for j in i.split(': ')]
            if typ in cols :
                deviceTest_df.at[ind,typ] = val
        except:
            print(i)

"browser": "subjectAgent: NoticiasBoom"





In [26]:
freq = deviceTrain_df.browser.value_counts()
browsers = freq[ freq > 2000].index.tolist()

deviceTrain_df.loc[~deviceTrain_df.browser.isin(browsers), 'browser'] = 0
deviceTest_df.loc[~deviceTest_df.browser.isin(browsers), 'browser'] = 0

# fix some str/int problem
deviceTrain_df = deviceTrain_df.astype(dtype=np.str)
deviceTest_df = deviceTest_df.astype(dtype=np.str)


lencoder = LabelEncoder()

_ = lencoder.fit(pd.concat((deviceTrain_df.browser, deviceTest_df.browser)))
deviceTrain_df.browser = lencoder.transform(deviceTrain_df.browser)
deviceTest_df.browser = lencoder.transform(deviceTest_df.browser)

_ = lencoder.fit(pd.concat((deviceTrain_df.operatingSystem, deviceTest_df.operatingSystem)))
deviceTrain_df.operatingSystem = lencoder.transform(deviceTrain_df.operatingSystem)
deviceTest_df.operatingSystem = lencoder.transform(deviceTest_df.operatingSystem)

_ = lencoder.fit(pd.concat((deviceTrain_df.deviceCategory, deviceTest_df.deviceCategory)))
deviceTrain_df.deviceCategory = lencoder.transform(deviceTrain_df.deviceCategory)
deviceTest_df.deviceCategory = lencoder.transform(deviceTest_df.deviceCategory)


In [27]:
new_train = pd.concat((new_train,deviceTrain_df),sort=False,axis=1)
new_train.drop(columns=['device'], inplace=True)

new_test = pd.concat((new_test,deviceTest_df),sort=False,axis=1)
new_test.drop(columns=['device'], inplace=True)


-----------------------------------

## Last touch [ target, drop others..]

In [35]:
fullVisitorId = new_test.fullVisitorId.copy()


cols_to_drop = ['fullVisitorId','sessionId','visitId','visitStartTime']
new_train.drop(columns=cols_to_drop, inplace=True)
new_test.drop(columns=cols_to_drop, inplace=True)

new_train.rename(columns={'transactionRevenue':'target'}, inplace=True)

new_train.to_hdf('./ready_train.hdf',key='latest')
new_test.to_hdf('./ready_test.hdf',key='latest')
fullVisitorId.to_csv('./fullVisitorId.csv',index=False)

---------------------

## Check correlation : 

In [30]:
corr = new_train.corr()

for i in corr.columns:
    for j, val in zip(corr[i].index, corr[i].values):
        if i == j : continue
        if np.abs(val) > .5:
            print (i,' & ',j, ' : ', val)

channelGrouping  &  medium  :  0.8207965760842164
visitId  &  visitStartTime  :  0.9999999999903184
visitId  &  year  :  0.8630703172955572
visitId  &  month  :  -0.5023355574379903
visitId  &  dayOfYear  :  -0.5006630265278692
visitStartTime  &  visitId  :  0.9999999999903184
visitStartTime  &  year  :  0.8630703172030794
visitStartTime  &  month  :  -0.5023355577686512
visitStartTime  &  dayOfYear  :  -0.5006630259263771
hits  &  pageviews  :  0.9832010020182913
newVisits  &  isTrueDirect  :  -0.6160405810324063
pageviews  &  hits  :  0.9832010020182913
medium  &  channelGrouping  :  0.8207965760842164
medium  &  isTrueDirect  :  -0.6177875335253851
isTrueDirect  &  newVisits  :  -0.6160405810324063
isTrueDirect  &  medium  :  -0.6177875335253851
isVideoAd  &  slot  :  0.998676850320491
slot  &  isVideoAd  :  0.998676850320491
year  &  visitId  :  0.8630703172955572
year  &  visitStartTime  :  0.8630703172030794
year  &  month  :  -0.8682717205928565
year  &  dayOfYear  :  -0.8693274

In [32]:
corr['target']


channelGrouping      -0.006644
visitId               0.002724
visitNumber           0.051366
visitStartTime        0.002724
bounces              -0.032206
hits                  0.154333
newVisits            -0.041164
pageviews             0.155589
transactionRevenue    1.000000
medium               -0.008569
isTrueDirect          0.030819
isVideoAd             0.000834
slot                  0.000859
year                  0.003194
month                -0.002849
dayOfYear            -0.002817
continent            -0.025523
subContinent         -0.009144
country               0.022372
browser              -0.014879
operatingSystem      -0.011925
deviceCategory       -0.015580
Name: transactionRevenue, dtype: float64

In [37]:
cols_to_drop = ['month', 'slot', 'visits']
new_train.drop(columns=cols_to_drop, inplace=True)
new_test.drop(columns=cols_to_drop, inplace=True)


new_train.to_hdf('./ready_train.hdf',key='latest')
new_test.to_hdf('./ready_test.hdf',key='latest')

------------------------------

In [39]:
new_train.shape

(903653, 18)